
# Portfolio Optimization with the Quantum Approximate Optimization Algorithm (QAOA)



## Introduction

Portfolio optimization [[1](#PortfolioWiki)] is the process of allocating a portfolio of financial assets optimally, according to some predetermined goal. Usually, the goal is to maximize the potential return while minimizing the financial risk of the portfolio. One can express this problem as a combinatorial optimization problem. In this demo, we'll show how the Quantum Approximate Optimization Algorithm (QAOA) [[2](#QAOA)] can be used with the Classiq platform to solve the portfolio optimization problem.

### Modeling the Portfolio Optimization Problem

As a first step, we have to model the problem mathematically. We will use a simple yet powerful model, which captures the essence of portfolio optimization:

* A portfolio is built from a pool of $n$ financial assets, each asset labeled $i \in \{1,\ldots,n\}$.
* Every asset's return is a random variable, with expected value $\mu_i$ and variance $\Sigma_i$ (modeling the financial risk involved in the asset).
* Every two assets $i \neq j$ have covariance $\Sigma_{ij}$ (modeling market correlation between assets).
* Every asset $i$ has a weight $w_i \in D_i = \{0,\ldots,b_i\}$ in the portfolio, with $b_i$ defined as the budget for asset $i$ (modeling the maximum allowed weight of the asset).
* The return vector $\mu$, the covariance matrix $\Sigma$ and the weight vector $w$ are defined naturally from the above (with the domain $D = D_1 \times D_2 \times \ldots \times D_n$ for $w$).

With the above definitions, the total expected return of the portfolio is $\mu^T w$ and the total risk is $w^T \Sigma w$. We'll use a simple difference of the two as our cost function, while setting a constraint that the total sum of assets does not exceed a predefined budget $B$. We note that there are many other possibilities for defining a cost function (e.g. add a scaling factor to the risk/return or even some non-linear relation). For reasons of simplicity we select the model below, and we assume all constants and variables are dimensionless.
Thus, the problem is, given the constant inputs $\mu, \Sigma, D, B$, to find optimal variable $w$ as follows:

$\begin{equation*}
\min_{w \in D}  w^T \Sigma w - \mu^T w,
\end{equation*}$
subject to $\Sigma_{i} w_i \leq B$.

The case presented above is called integer portfolio optimization, since the domains $D_i$ are over the (positive) integers.
Another variation of this problem defines weights over binary domains, and will not be discussed here.



## Setting up the problem

With the mathematical definition in place, we begin the implementation by importing necessary packages and classes. We will use the following external dependencies:

1. NumPy
2. Matplotlib
3. Pyomo - a python framework for modeling optimization problems, which the Classiq platform uses as an interface to these types of problems.

From the `classiq` package, we require classes related to combinatorial optimization and QAOA.

In [1]:
'''
!pip install -q numpy==1.22.3
!pip install -q matplotlib==3.5.1
!pip install -q pyomo==6.0.1
pip install -u classiq
pip install -u classiq-interface
'''


'\n!pip install -q numpy==1.22.3\n!pip install -q matplotlib==3.5.1\n!pip install -q pyomo==6.0.1\npip install -u classiq\npip install -u classiq-interface\n'

In [2]:
## This demo was tested on Classiq version 0.13.0

import numpy as np
import pyomo.core as pyo
from classiq import CombinatorialOptimization
from classiq.interface.combinatorial_optimization.preferences import QAOAPreferences
from classiq.interface.combinatorial_optimization.solver_types import QSolver
from classiq.interface.executor.optimizer_preferences import CombinatorialOptimizer
from classiq.interface.executor.vqe_result import OptimizationResult
import classiq

classiq.authenticate()

C:\Users\paulgriffin\Anaconda3\lib\site-packages\classiq\_internals\authentication\token_manager.py:73: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


## The Portfolio Optimization Problem Parameters

First we define the parameters of the optimization problem, which include the expected return vector, the covariance matrix, the total budget and the asset-specific budgets:

In [16]:
#Add one more asset
returns = np.array([3, 4, -1, 3]) #3, 4, -1, 
# fmt: off
covariances = np.array(
    [
        [ 0.9,  0.5, -0.7, 0.4],
        [ 0.5,  0.9, -0.2, 0.3],
        [-0.7, -0.2,  0.9, 0.2],
        [ 0.4,  0.3,  0.2, 0.9],
        
    ]
)
# fmt: on
total_budget = 8 #6
specific_budgets = [2, 3, 2, 1]
print(returns)
print(covariances)

[ 3  4 -1  3]
[[ 0.9  0.5 -0.7  0.4]
 [ 0.5  0.9 -0.2  0.3]
 [-0.7 -0.2  0.9  0.2]
 [ 0.4  0.3  0.2  0.9]]


In [17]:
'''
#Original parameters
returns = np.array([3, 4, -1]) 
# fmt: off
covariances = np.array(
    [
        [ 0.9,  0.5, -0.7],
        [ 0.5,  0.9, -0.2],
        [-0.7, -0.2,  0.9],
               
    ]
)
# fmt: on
total_budget = 6 
specific_budgets = [2, 3, 2]
print(returns)
print(covariances)
'''

'\n#Original parameters\nreturns = np.array([3, 4, -1]) \n# fmt: off\ncovariances = np.array(\n    [\n        [ 0.9,  0.5, -0.7],\n        [ 0.5,  0.9, -0.2],\n        [-0.7, -0.2,  0.9],\n               \n    ]\n)\n# fmt: on\ntotal_budget = 6 \nspecific_budgets = [2, 3, 2]\nprint(returns)\nprint(covariances)\n'

## The Pyomo Model for the Problem

We proceed by defining the Pyomo model that will be used on the Classiq platform, using the problem parameters defined above:

In [18]:
model = pyo.ConcreteModel()
num_assets = len(returns)

# setting the variables
model.w = pyo.Var(
    range(num_assets),
    domain=pyo.Integers,
    bounds=lambda _, idx: (0, specific_budgets[idx]),
)
w_array = model.w.values()

# setting the constraint
model.budget_rule = pyo.Constraint(expr=(sum(w_array) <= total_budget))

# setting the expected return and risk
model.expected_return = returns @ w_array
model.risk = w_array @ covariances @ w_array

# setting the cost function to minimize
model.cost = pyo.Objective(expr=model.risk - model.expected_return, sense=pyo.minimize)

## Setting Up the Classiq Problem Instance

In order to solve the Pyomo model defined above, we use the Classiq combinatorial optimization engine. For the quantum part of the QAOA algorithm (`QAOAPreferences`) we define the type of quantum solver (`qsolver`) to use and the number of repetitions (`qaoa_reps`):

In [19]:
qaoa_preferences = QAOAPreferences(qaoa_reps=5, qsolver=QSolver.QAOAMixer)

For the classical optimization part of the QAOA algorithm (`CombinatorialOptimizer`) we define the number of times we measure the quantum circuit for each classical iteration (`num_shots`), the maximum number of classical iterations (`max_iteration`) and the $\alpha$-parameter (`alpha_cvar`) for running CVaR-QAOA, an improved variation of the QAOA algorithm [[3](#cvar)]:

In [20]:
optimizer_preferences = CombinatorialOptimizer(
    num_shots=1000, max_iteration=60, alpha_cvar=0.7
)

Last, we define the `CombinatorialOptimization` instance which we can use to solve the problem:

In [21]:
problem = CombinatorialOptimization(
    model=model,
    qsolver_preferences=qaoa_preferences,
    optimizer_preferences=optimizer_preferences,
)

## Generating the QAOA Circuit and Solving the Problem

We can now generate and view the QAOA circuit (ansatz) used to solve the optimization problem. The generation should typically take less than a minute for the above preferences:

In [22]:
circuit_result = problem.generate()

#circuit_result.show_interactive()
print("Generation complete")

Solving failed: all vars must be from the same type



ClassiqGenerationError: Solving failed: all vars must be from the same type

We can also solve the problem using the generated circuit by using the `CombinatorialOptimization.solve()` method (taking approximately 3 minutes):

In [ ]:
optimization_result = problem.solve()
print("Solved")

We can now print the optimization results:

In [ ]:
optimization_result.num_solutions = 3
print(optimization_result)

And the histogram:

In [ ]:
optimization_result.histogram()

Lastly, we can compare the quantum solution to the classical solution of the problem:

In [ ]:
optimization_classical_result = problem.solve_classically()

print(optimization_classical_result)

We can see that most of the solutions obtained by running QAOA are close to the minimal solution obtained classically, demonstrating the effectiveness of the algorithm. Also, note the non-trivial solution which includes a non-zero weight for the asset with negative expected return, demonstrating that it sometimes makes sense to include such assets in the portfolio as a risk-mitigation strategy - especially if they are highly anti-correlated with the rest of the assets.


## References

<a id='PortfolioWiki'>[1]</a>: [Portfolio Optimization (Wikipedia)](https://en.wikipedia.org/wiki/Portfolio_optimization)

<a id='QAOA'>[2]</a>: [Farhi, Edward, Jeffrey Goldstone, and Sam Gutmann. "A quantum approximate optimization algorithm." arXiv preprint arXiv:1411.4028 (2014).](https://arxiv.org/abs/1411.4028)

<a id='cvar'>[3]</a>: [Barkoutsos, Panagiotis Kl, et al. "Improving variational quantum optimization using CVaR." Quantum 4 (2020): 256.](https://arxiv.org/abs/1907.04769)